**Loading Libraries**

In [1]:
# Load libraries
import datetime as dt
from datetime import datetime
import time 
import calendar
start_time = datetime.now()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import scipy.stats as sps
import random

print('LOAD DURATION:',datetime.now() - start_time)

LOAD DURATION: 0:00:01.296221


**Load Data**

In [4]:
%%time
df = pd.read_csv('candidate_var_hw7.csv', index_col=0)

CPU times: user 28.7 s, sys: 4.46 s, total: 33.1 s
Wall time: 33.3 s


In [5]:
df['date'] = pd.to_datetime(df.date)

df1 = df.copy()
final = df.copy()

df1['check_date'] = df1.date
df1['check_record'] = df1.record

In [6]:
attributes=['ssn','address','homephone','name_dob']


begin = time.time()
for entity in attributes:
    for field in attributes:
        st = time.time()
        if field == entity:
            continue
        else:
            df_c = df1[['record', 'date', entity]]
            df_d = df1[['check_record', 'check_date', entity,field]]
            temp = pd.merge(df_c, df_d, left_on=entity, right_on=entity)

        # number of unique for particular field
        for offset_t in [1, 3, 7, 14, 30]:
            count_day_df = temp[(temp.check_date >= (temp.date - dt.timedelta(offset_t)))
                                & (temp.record >= temp.check_record)]
            col_name = f'{entity}_unique_count_for_{field}_{offset_t}'
            mapper = count_day_df.groupby(['record'])[field].nunique()
            final[col_name] = final.record.map(mapper)

        print(f'Run time for entity {entity} in field {field}------------- {time.time() - st}s')

    print(f'Total run time: {(time.time() - begin) / 60}min')

Run time for entity ssn in field address------------- 7.378212213516235s
Run time for entity ssn in field homephone------------- 4.73783802986145s
Run time for entity ssn in field name_dob------------- 5.363019704818726s
Total run time: 0.2913571000099182min
Run time for entity address in field ssn------------- 5.159642934799194s
Run time for entity address in field homephone------------- 4.955261945724487s
Run time for entity address in field name_dob------------- 5.405716180801392s
Total run time: 0.5500397006670634min
Run time for entity homephone in field ssn------------- 19.802448749542236s
Run time for entity homephone in field address------------- 18.062894105911255s
Run time for entity homephone in field name_dob------------- 17.94045901298523s
Total run time: 1.4801429152488708min
Run time for entity name_dob in field ssn------------- 5.712926149368286s
Run time for entity name_dob in field address------------- 5.380512952804565s
Run time for entity name_dob in field homephone

In [7]:
final.to_csv('400_var.csv')

In [5]:
# add a random variable
randomlist = []
for i in range(1000000):
    n = random.randint(1,10)
    randomlist.append(n)

final['random_var'] = randomlist

final = final.drop(columns=['record'])

In [6]:
%%time
# create oot train_test data
train_test = final[(final.date > '2016-01-14')&(final.date < '2016-11-01')]

CPU times: user 968 ms, sys: 846 ms, total: 1.81 s
Wall time: 1.81 s


In [7]:
# create the output dataframe
KSFDR = pd.DataFrame(final.columns, columns=['Variable'])

KSFDR.reset_index(inplace=True)
KSFDR.rename(columns={"index":"Variable Number"}, inplace=True)

In [8]:
# separate the good and bad
goods = train_test[train_test.fraud_label == 0].copy()
bads = train_test[train_test.fraud_label == 1].copy()

In [9]:
# calculate the KS score
KSFDR['ks'] = np.NaN
cols = train_test.columns

i = 0
for column in cols:
    KSFDR.loc[i,'ks'] = sps.ks_2samp(goods[column],bads[column])[0]
    i = i+1

In [10]:
KSFDR['rank_ks'] = KSFDR['ks'].rank(ascending = True)
KSFDR.sort_values(by=['rank_ks'], ascending=False, inplace=True)

In [48]:
KSFDR.reset_index(drop=True, inplace=True)

In [51]:
# calculate FDR @3%
KSFDR['FDR'] = np.NaN
listvars = list(KSFDR["Variable"])
listvars = listvars[1:]

j = 1
topRows = int(round(len(train_test)*0.03))
numbads = len(train_test[train_test.fraud_label==1])

for column in listvars:
    temp = train_test[[column,'fraud_label']].copy() 
    temp0 = temp.sort_values(column,ascending=False)
    temp1 = temp0.head(topRows)
    temp2 = temp0.tail(topRows)
    needed1 = temp1.loc[:,'fraud_label']
    needed2 = temp2.loc[:,'fraud_label']
    FDR1 = sum(needed1)/numbads
    FDR2 = sum(needed2)/numbads
    FDRate = np.maximum(FDR1,FDR2)
    KSFDR.loc[j, 'FDR'] = FDRate
    j = j + 1

In [53]:
KSFDR.loc[0, 'FDR'] = 1

In [59]:
KSFDR['rank_FDR'] = KSFDR['FDR'].rank(ascending = True)

In [63]:
KSFDR['average_rank'] = (KSFDR['rank_ks'] + KSFDR['rank_FDR']) / 2
KSFDR.sort_values(by=['average_rank'], ascending=False, inplace=True)
KSFDR.reset_index(drop=True, inplace=True)

In [64]:
KSFDR

Variable Number                          Variable        ks  rank_ks  \
0                  8                       fraud_label  1.000000    420.0   
1                 36                 address_day_since  0.334096    419.0   
2                 64             fulladdress_day_since  0.333210    418.0   
3                 42                  address_count_30  0.332725    417.0   
4                 70              fulladdress_count_30  0.332032    416.0   
..               ...                               ...       ...      ...   
415              372  ssn_unique_count_for_name_dob_14  0.000995      6.0   
416              371   ssn_unique_count_for_name_dob_7  0.000573      4.0   
417              369   ssn_unique_count_for_name_dob_1  0.000203      1.0   
418              370   ssn_unique_count_for_name_dob_3  0.000287      3.0   
419              404   name_dob_unique_count_for_ssn_1  0.000250      2.0   

          FDR  rank_FDR  average_rank  
0    1.000000     420.0        420.00  
1    0.355302     418.0        418.50  
2    0.358349     419.0        418.50  
3    0.353300     416.0        416.50  
4    0.354954     417.0        416.50  
..        ...       ...           ...  
415  0.033432      25.5         15.75  
416  0.033606      27.0         15.50  
417  0.033432      25.5         13.25  
418  0.033258      23.0         13.00  
419  0.033345      24.0         13.00  

[420 rows x 7 columns]

In [65]:
# KSFDR.to_excel('ksfdr_score_ranked.xlsx')